In [2]:
!pip install tensorflow

  Using cached tensorflow-2.8.0-cp37-cp37m-manylinux2010_x86_64.whl (497.5 MB)
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
You should consider upgrading via the '/home/mohammad/.pyenv/versions/3.7.12/envs/cv_noor/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import os
import dlib
import cv2
import time
import numpy as np
from PIL import Image
from scipy.spatial import distance as dist
from collections import OrderedDict
import tensorflow as tf


gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)


label_map= ["biscuit_a","biscuit_b","biscuit_c"]
label_map.sort()
detect_fn=tf.saved_model.load("./saved_model")

2022-03-02 23:09:28.693722: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-02 23:09:28.867506: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-02 23:09:28.868237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-02 23:09:30.991775: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
detect_fn

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f1cfa1f8d90>

In [5]:
import math
from tensorflow.keras.applications import resnet50
from tensorflow.keras.layers import Dense, Conv2D, Flatten 
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
 
# load class model
class_model = load_model('./imageacc_model.h5')

#Backbone_SNN
# base_resnet=tf.keras.applications.ResNet50(include_top=False,weights="imagenet",input_shape=(150,150,3),classes=len(label_map))
# # for i in base_resnet.layers[:(math.ceil(len(base_resnet.layers)/2))]:
# #     i.trainable=False
# conv1=Conv2D(48,(1,1),padding="valid",activation="relu")
# dr1=layers.Dropout(0.3)
# avgpool1 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))
# conv2=Conv2D(78,(1,1),padding="same",activation="relu")
# dr2=layers.Dropout(0.2)
# flatten_layer = Flatten()
# dense1 = Dense(108,activation='sigmoid')

# embedding_model=tf.keras.Sequential([base_resnet,conv1,avgpool1,conv2,flatten_layer,dense1])
# embedding_model.load_weights(".\\checkpoint_94_0.00.hdf5")

In [6]:
class_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (None, 108)               23723690  
                                                                 
 dense_9 (Dense)             (None, 3)                 327       
                                                                 
Total params: 23,724,017
Trainable params: 327
Non-trainable params: 23,723,690
_________________________________________________________________


In [7]:
def detect_boxes(frame, detect_fn,thresh,H,W):
        image_np = np.array(frame)
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.uint8)
        detections = detect_fn(input_tensor)
        # detection_classes should be ints.
        score_mask=(detections['detection_scores']>0.5).numpy().reshape(-1)
        rects_masked=detections['detection_boxes'].numpy().reshape(100,4)[score_mask]
        rects=(rects_masked*np.array([H,W,H,W])).astype(int)
        func=lambda x: (x[1]+x[3])/2
        rect_mask=np.array(list(map(func,rects)))>600*0.30
        print(rects)
        print(rect_mask)
        valid_rects=rects[rect_mask]
        print("Valid_rects\n {}".format(valid_rects))
        return valid_rects
    
def visualize_boxes(image,tracks):    
    for id_,track in tracks.items():
        (startY,startX,endY,endX) = track.box
        name="unknown" if track.name==None else track.name
        # draw the bounding box of the face along with the associated probability
        cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
        cv2.putText(image, name, (startX, startY),cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 255, 255), 2)
        #cv2.putText(image, name, (startX, startY),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255,255), 2) 
    return image

In [8]:


# def get_name(model,svm,face_pixels,label_map):
#     img=cv2.resize(face_pixels,(160,160))
#     img = np.around(np.array(img) / 255.0, decimals=12)
#     x_train = np.expand_dims(img, axis=0)
#     embedding = model.predict(x_train)
#     embedding= embedding / np.linalg.norm(embedding, ord=2)
#     name=svm.predict(embedding.reshape(1,-1))
#     return label_map[name]



class Track():
    def __init__(self,tracker,box):
        self.tracker=tracker
        self.box=box
        self.recognized_face=False
        self.encodings=None 
        self.name=None
        self.prob_face={}
        self.centroid=self.get_centroid()
        self.rec_count=0
        #self.spoof_confirm=[]


    def update_track(self,frame):
        self.tracker.update(frame)
        pos=self.tracker.get_position()
        startX = int(pos.left())
        startY = int(pos.top())
        endX = int(pos.right())
        endY = int(pos.bottom())
        tracked_box=np.array([startY,startX,endY,endX])
        print("tracked_box")
        #print(("tracked_box",tracked_box))
        prev_box=np.array(self.box)
        #self.box=tracked_box.astype('int')
        return tracked_box
        #print(("prev_box",prev_box))
        #self.box=self.box if get_overlap(tracked_box,prev_box)>0.95 else tracked_box.astype('int')
        
    def box_recognize(self,image,class_model,label_map,tracker):
        global W,H
        if (self.name==None and 0.46*W<self.get_centroid()[1]<0.50*W):
            starty,startx,endy,endx=self.box
            box_roi = image[starty:endy,startx:endx]  
            img=cv2.cvtColor(box_roi,cv2.COLOR_BGR2RGB)
            im = Image.fromarray(img, 'RGB')
            #Resizing into dimensions you used
            im = im.resize((150,150))
            img_array = (np.array(im).astype(np.float32)/255)
            #Expand dimensions to match the 4 D Tensor shape.
            img_array = np.expand_dims(img_array, axis = 0)
            idx_=class_model.predict(img_array)
            name=label_map[np.argmax(idx_)]
            self.name=name
            tracker.count[name]=tracker.count.get(name,0)+1
            print(tracker.count)
    def get_box(self):
        pos=self.tracker.get_position()
        return [int(pos.left()),int(pos.top()),int(pos.right()),int(pos.bottom())]


    def get_centroid(self):
        box=self.box
        return (int((box[0]+box[2])/2),int((box[1]+box[3])/2))


class Tracker():
    
    global frame
    
    def __init__(self, maxDisappeared=5,maxDistance=150,thresh=None):
        # initialize the next unique object ID along with two ordered
        # dictionaries used to keep track of mapping a given object
        # ID to its centroid and number of consecutive frames it has
        # been marked as "disappeared", respectively
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        self.count={}
        self.thresh=thresh
        # store the number of maximum consecutive frames a given
        # object is allowed to be marked as "disappeared" until we
        # need to deregister the object from tracking
        self.maxDisappeared = maxDisappeared
        self.maxDistance=maxDistance

    def register(self, rect,frame):
    
        # when registering an object we use the next available object
        # ID to store the centroid
        t = dlib.correlation_tracker()
        (startY, startX, endY, endX)=rect
        drect = dlib.rectangle(startX, startY, endX, endY)
        t.start_track(frame, drect)
        name='obj_'+str(self.nextObjectID)
        self.objects[name] = Track(t,rect)
        self.disappeared[name] = 0
        self.nextObjectID+=1

    def deregister(self, objectID):
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
        trk=self.objects[objectID]
        startY,startX,endY,endX=trk.box
        print("vanishing_arr: {}".format(trk.box))
#         if int(startX/2+endX/2) <self.thresh:     
#             name=trk.name
#             self.count[name]=self.count.get(name,0)+1
        del self.objects[objectID]
        del self.disappeared[objectID]
        
    def deregister(self, objectID):
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
#         trk=self.objects[objectID]
#         startY,startX,endY,endX=trk.box
#         if int(startX/2+endX/2) <self.thresh:     
#             name=trk.name
#             self.count[name]=self.count.get(name,0)+1
            
        del self.objects[objectID]
        del self.disappeared[objectID]
        
    def track_update(self,frame):
        height,width,channel = frame.shape
        for objectID in list(self.disappeared.keys()):
            trk=self.objects[objectID]
            startY,startX,endY,endX=trk.box
            if int(startX/2+endX/2)<(0.35*width):
                self.disappeared[objectID]+=4.7
            if self.disappeared[objectID]>self.maxDisappeared:
                self.deregister(objectID)
        t_rects=[]
        for name in self.objects.keys():
            t_rect=self.objects[name].update_track(frame)
            t_rects.append(t_rect)
        return np.array(t_rects)


    def update(self,frame, coords,thresh):
        
        for objectID in list(self.disappeared.keys()):
            trk=self.objects[objectID]
            if trk.centroid[0]<thresh:
                self.disappeared[objectID]+=1.7
        
        
        
        if len(coords) == 0:
            # loop over any existing tracked objects and mark them
            # as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1

                # if we have reached a maximum number of consecutive
                # frames where a given object has been marked as
                # missing, deregister it
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)

            # return early as there are no centroids or tracking info
            # to update
            return self.objects
        inputCentroids = np.zeros((len(coords), 2), dtype="int")

        # loop over the bounding box rectangles
        for (i, (startY, startX, endY, endX)) in enumerate(coords):
            # use the bounding box coordinates to derive the centroid
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)

        # initialize an array of input centroids for the current frame
        
        # if we are currently not tracking any objects take the input
        # centroids and register each of them

        if len(self.objects) == 0:
            for i in range(0, len(coords)):
                self.register(coords[i],frame)
            
            
        # otherwise, are are currently tracking objects so we need to
        # try to match the input centroids to existing object
        # centroids
        else:
            # grab the set of object IDs and corresponding centroids
            object_copy=self.objects.copy()
            tracks_copy=list(object_copy.values())
            #print('tracks:'+str(tracks_copy))
            objectIDs = list(object_copy.keys())
            #print('objectIDs:' + str(objectIDs))
            objectCentroids = [i.centroid for i in tracks_copy]

            # compute the distance between each pair of object
            # centroids and input centroids, respectively -- our
            # goal will be to match an input centroid to an existing
            # object centroid
            D = dist.cdist(np.array(objectCentroids), inputCentroids)

            # in order to perform this matching we must (1) find the
            # smallest value in each row and then (2) sort the row
            # indexes based on their minimum values so that the row
            # with the smallest value as at the *front* of the index
            # list
            rows = D.min(axis=1).argsort()
            # next, we perform a similar process on the columns by
            # finding the smallest value in each column and then
            # sorting using the previously computed row index list
            cols = D.argmin(axis=1)[rows]
            
            # in order to determine if we need to update, register,
            # or deregister an object we need to keep track of which
            # of the rows and column indexes we have already examined
            usedRows = set()
            usedCols = set()

            # loop over the combination of the (row, column) index
            # tuples
            for (row, col) in zip(rows, cols):
                # if we have already examined either the row or
                # column value before, ignore it
                # val
                if row in usedRows or col in usedCols:
                    continue
                # if the distance between centroids is greater than
                # the maximum distance, do not associate the two
                # centroids to the same object
                if D[row, col] > self.maxDistance:
                    continue

                # otherwise, grab the object ID for the current row,
                # set its new centroid, and reset the disappeared
                # counter
                objectID = objectIDs[row]
                prev_name=self.objects[objectID].name
                prev_recog_state=self.objects[objectID].recognized_face
                t = dlib.correlation_tracker()
                (y1,x1,y2,x2)=coords[col]
                #name=names[col]
                rect=coords[col]
                drect = dlib.rectangle(int(x1),int(y1),int(x2),int(y2))
                t.start_track(frame, drect)
                n_track=Track(t,rect)
                self.objects[objectID] = n_track
                self.objects[objectID].name=prev_name
                self.objects[objectID].recognized_face=prev_recog_state
                #self.objects[objectID].box = coords[col]
                self.disappeared[objectID] = 0

                # indicate that we have examined each of the row and
                # column indexes, respectively
                usedRows.add(row)
                usedCols.add(col)

            # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            # in the event that the number of object centroids is
            # equal or greater than the number of input centroids
            # we need to check and see if some of these objects have
            # potentially disappeared
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    # check to see if the number of consecutive
                    # frames the object has been marked "disappeared"
                    # for warrants deregistering the object
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)

            # otherwise, if the number of input centroids is greater
            # than the number of existing object centroids we need to
            # register each new input centroid as a trackable object
            else:
                for col in unusedCols:
                    self.register(coords[col],frame)

        # return the set of trackable objects
        return self.objects


In [9]:
import imutils
import time
from PIL import Image
cap=cv2.VideoCapture("./testing_videos/short_6.mp4")
#if vid_path:
    #cap=cv2.VideoCapture(r'G:\Wondershare UniConverter\Converted\ch01_20210707001338.mp4')
#else:

fps = cap.get(cv2.CAP_PROP_FPS)
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#out = cv2.VideoWriter('full_output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), fps, (W,H))
fc=0
thresh=int(600*0.30)
tic=time.time()
name_thresh=600*0.50
tracker=Tracker(maxDisappeared=2,maxDistance=150,thresh=thresh)
func=lambda x: (x[1]+x[3])/2
FPS=[]
while True:
    ret, frame = cap.read()
    if not ret:
        break
    start=time.time()
    img=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    im = Image.fromarray(img, 'RGB')
            #Resizing into dimensions you used
    im = im.resize((337,600))
    img_array = (np.array(im).astype(np.float32))
            #Expand dimensions to match the 4 D Tensor shape.
    
    frame=imutils.resize(frame, width=600)
    H,W=frame.shape[:2]
    print("height: {}, width: {}".format(H,W))
    if (fc==0 or fc%4==0):
        rects=detect_boxes(img_array, detect_fn,thresh,H,W)
#         rect_mask=np.array(list(map(func,rects)))>thresh
#         print(rects)
#         print(rect_mask)
#         valid_rects=rects[rect_mask]
        tracker.objects=tracker.update(frame,rects,thresh)
       
    else:
        t_rects=tracker.track_update(frame)
#         rect_mask=np.array(list(map(func,t_rects)))>thresh
#         print(rects)
#         print(rect_mask)
#         valid_rects=t_rects[rect_mask]  
#         print("tracked_valids: \n {}".format(valid_rects))
        tracker.objects=tracker.update(frame,t_rects,thresh)
            
    #Performing Liveness test for antispoofing and 
    #implementing face recognition per face tracked
    for i in tracker.objects.values():
        i.box_recognize(frame,class_model,label_map,tracker)
    image=visualize_boxes(frame,tracker.objects)
    y_stat=20
    for item,count in tracker.count.items():
        cv2.putText(image,'{}: {}'.format(item,count),(0,y_stat),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        y_stat+=20
    blk = np.zeros(image.shape, np.uint8)
    cv2.rectangle(blk, (int(0.45*W),0), (int(0.55*W), 337), (0, 255, 0), -1)
    res = cv2.addWeighted(image,1, blk, 0.25, 1)

    # Putting the image back to its position
    res=imutils.resize(res,width=1200)
    cv2.imshow('faces',res)
    fc+=1
    end=time.time()
    print("FPS: {}".format(1/(end-start)))
    FPS.append(int(1/(end-start)))
    print(tracker.objects.keys())
    if cv2.waitKey(10) & 0xff==ord('q'):
        print("STOPPING APPLICATION")
        
        break
print("MEAN FPS THROUGHOUT VIDEO: {}".format(np.array(FPS).sum()/len(FPS)))        
cap.release()
cv2.destroyAllWindows()

height: 337, width: 600


2022-03-02 23:10:01.842960: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302
2022-03-02 23:10:08.172698: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-03-02 23:10:08.739114: W tensorflow/core/common_runtime/bfc_allocator.cc:343] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.


[[122 464 244 592]
 [ 98 492 135 600]]
[ True  True]
Valid_rects
 [[122 464 244 592]
 [ 98 492 135 600]]
FPS: 0.05628543585039197
odict_keys(['obj_0', 'obj_1'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 2.684854587318342
odict_keys(['obj_0'])
height: 337, width: 600
tracked_box
FPS: 15.919535125574546
odict_keys(['obj_0'])
height: 337, width: 600
tracked_box
FPS: 20.99198718750782
odict_keys([])
height: 337, width: 600
[[130 450 252 585]]
[ True]
Valid_rects
 [[130 450 252 585]]
FPS: 10.16936561238468
odict_keys(['obj_2'])
height: 337, width: 600
tracked_box
FPS: 23.27569769313156
odict_keys(['obj_2'])
height: 337, width: 600
tracked_box
FPS: 23.03410931896689
odict_keys(['obj_2'])
height: 337, width: 600
tracked_box
FPS: 23.260208184293564
odict_keys([])
height: 337, width: 600
[[140 430 260 578]]
[ True]
Valid_rects
 [[140 430 260 578]]
FPS: 17.90203677462312
odict_keys(['obj_3'])
height: 337, width: 600
tracked_box
FPS: 22.3641346656287
odict_keys(['obj_3'])
height: 337, 

2022-03-02 23:10:18.260231: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.08GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-03-02 23:10:18.260268: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.08GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-03-02 23:10:18.271346: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-03-02 23:10:18.271374: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Alloc

{'biscuit_b': 1}
FPS: 0.6622750831928693
odict_keys(['obj_12'])
height: 337, width: 600
[[178 200 299 367]
 [163 371 276 507]
 [256 260 334 483]]
[ True  True  True]
Valid_rects
 [[178 200 299 367]
 [163 371 276 507]
 [256 260 334 483]]
FPS: 14.30843809166425
odict_keys(['obj_12', 'obj_14', 'obj_15'])
height: 337, width: 600
tracked_box
tracked_box
tracked_box
FPS: 12.391805574431123
odict_keys(['obj_12', 'obj_14', 'obj_15'])
height: 337, width: 600
tracked_box
tracked_box
tracked_box
FPS: 15.978727051768999
odict_keys(['obj_12', 'obj_14', 'obj_15'])
height: 337, width: 600
tracked_box
tracked_box
tracked_box
FPS: 14.94933135639132
odict_keys(['obj_12', 'obj_15'])
height: 337, width: 600
[[170 326 288 473]
 [181 153 305 322]
 [270 240 335 440]]
[ True  True  True]
Valid_rects
 [[170 326 288 473]
 [181 153 305 322]
 [270 240 335 440]]
FPS: 13.609695475120464
odict_keys(['obj_12', 'obj_15', 'obj_16'])
height: 337, width: 600
tracked_box
tracked_box
tracked_box
FPS: 14.581326547285059
odi

...

height: 337, width: 600
tracked_box
FPS: 21.320645571228873
odict_keys(['obj_45'])
height: 337, width: 600
[[220 197 321 338]
 [166  41 326 235]]
[ True False]
Valid_rects
 [[220 197 321 338]]
FPS: 17.416976376252506
odict_keys(['obj_45'])
height: 337, width: 600
tracked_box
FPS: 20.644206112092768
odict_keys(['obj_45'])
height: 337, width: 600
tracked_box
FPS: 15.611882632760244
odict_keys(['obj_45'])
height: 337, width: 600
tracked_box
FPS: 18.479878043407382
odict_keys(['obj_45'])
height: 337, width: 600
[[223 159 325 299]
 [168  32 315 206]]
[ True False]
Valid_rects
 [[223 159 325 299]]
FPS: 16.46943519507445
odict_keys(['obj_45'])
height: 337, width: 600
tracked_box
FPS: 16.683123185235274
odict_keys(['obj_45'])
height: 337, width: 600
tracked_box
FPS: 18.849787876608904
odict_keys(['obj_45'])
height: 337, width: 600
FPS: 23.32099348905483
odict_keys([])
height: 337, width: 600
[[193 122 323 259]
 [171  18 306 186]]
[ True False]
Valid_rects
 [[193 122 323 259]]
FPS: 17.769387521

height: 337, width: 600
tracked_box
FPS: 16.92828400647377
odict_keys(['obj_63'])
height: 337, width: 600
tracked_box
{'biscuit_b': 3, 'biscuit_a': 3}
FPS: 10.78224477572037
odict_keys(['obj_63'])
height: 337, width: 600
tracked_box
FPS: 21.04950842872844
odict_keys(['obj_63'])
height: 337, width: 600
[[220 167 333 402]]
[ True]
Valid_rects
 [[220 167 333 402]]
FPS: 17.421244569235498
odict_keys(['obj_63'])
height: 337, width: 600
tracked_box
FPS: 17.784305660969373
odict_keys(['obj_63'])
height: 337, width: 600
tracked_box
FPS: 21.111595435715977
odict_keys(['obj_63'])
height: 337, width: 600
tracked_box
FPS: 20.897334462657565
odict_keys(['obj_63'])
height: 337, width: 600
[[224 131 331 354]
 [ 16 467 106 533]]
[ True  True]
Valid_rects
 [[224 131 331 354]
 [ 16 467 106 533]]
FPS: 16.920840090689776
odict_keys(['obj_63', 'obj_64'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 16.760388569875843
odict_keys(['obj_63'])
height: 337, width: 600
tracked_box
FPS: 21.780332653071824

FPS: 13.999072136389275
odict_keys(['obj_93', 'obj_95'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 18.69711004818815
odict_keys(['obj_93'])
height: 337, width: 600
[[240 232 331 392]
 [116 214 257 383]
 [157  42 214 127]]
[ True  True False]
Valid_rects
 [[240 232 331 392]
 [116 214 257 383]]
{'biscuit_b': 4, 'biscuit_a': 3}
FPS: 9.853208638434126
odict_keys(['obj_93', 'obj_96'])
height: 337, width: 600
tracked_box
tracked_box
{'biscuit_b': 5, 'biscuit_a': 3}
FPS: 10.006379379859386
odict_keys(['obj_93', 'obj_96'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 17.29417343223641
odict_keys(['obj_93', 'obj_96'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 18.497971280386693
odict_keys(['obj_93'])
height: 337, width: 600
[[244 190 334 347]
 [118 177 263 346]]
[ True  True]
Valid_rects
 [[244 190 334 347]
 [118 177 263 346]]
FPS: 15.546435772743447
odict_keys(['obj_93', 'obj_97'])
height: 337, width: 600
tracked_box
tracked_box
FPS: 19.1214263896677
odict_keys(['

height: 337, width: 600
tracked_box
FPS: 18.56396002443148
odict_keys([])
height: 337, width: 600
FPS: 26.523733036538633
odict_keys([])
height: 337, width: 600
FPS: 26.471170352418458
odict_keys([])
height: 337, width: 600
[[110 437 210 553]
 [102  82 171 143]
 [154  75 198 151]]
[ True False False]
Valid_rects
 [[110 437 210 553]]
FPS: 17.912587442452402
odict_keys(['obj_105'])
height: 337, width: 600
tracked_box
FPS: 23.240360160687075
odict_keys([])
height: 337, width: 600
FPS: 26.667921337241463
odict_keys([])
height: 337, width: 600
FPS: 26.616981850488642
odict_keys([])
height: 337, width: 600
[[118 430 220 553]
 [100  86 167 150]
 [154  82 192 154]]
[ True False False]
Valid_rects
 [[118 430 220 553]]
FPS: 18.041569167240194
odict_keys(['obj_106'])
height: 337, width: 600
tracked_box
FPS: 22.6314216186175
odict_keys([])
height: 337, width: 600
FPS: 26.47919191919192
odict_keys([])
height: 337, width: 600
FPS: 26.565563543085158
odict_keys([])
height: 337, width: 600
[[125 420 2

height: 337, width: 600
FPS: 25.68953077436623
odict_keys([])
height: 337, width: 600
[[189  33 256 143]]
[False]
Valid_rects
 []
FPS: 19.41879328864032
odict_keys([])
height: 337, width: 600
FPS: 18.86038302598623
odict_keys([])
height: 337, width: 600
FPS: 29.148162561850224
odict_keys([])
height: 337, width: 600
FPS: 24.569329810852132
odict_keys([])
height: 337, width: 600
[[186  29 251 132]]
[False]
Valid_rects
 []
FPS: 17.84574800770962
odict_keys([])
height: 337, width: 600
FPS: 26.63489845942822
odict_keys([])
height: 337, width: 600
FPS: 26.171870710096094
odict_keys([])
height: 337, width: 600
FPS: 25.96095616543494
odict_keys([])
height: 337, width: 600
[[184  27 242 127]]
[False]
Valid_rects
 []
FPS: 19.90330938529141
odict_keys([])
height: 337, width: 600
FPS: 26.649452308943502
odict_keys([])
height: 337, width: 600
FPS: 24.616627050503272
odict_keys([])
height: 337, width: 600
FPS: 20.814784671424174
odict_keys([])
height: 337, width: 600
[]
[]
Valid_rects
 []
FPS: 19.50